In [ ]:
import pandas as pd 

In [ ]:
df=pd.read_excel("Online Retail.xlsx")

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df['Country'].unique()

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
# how often customers buys
frequent_purchase=df.groupby("CustomerID")["InvoiceNo"].nunique().reset_index()
frequent_purchase

In [ ]:
frequent_purchase.max()

In [ ]:
# how much do customer buys
df["Total_price"]=df["Quantity"] * df["UnitPrice"]
df["Total_price"]

In [ ]:
spending_cost=df.groupby("CustomerID")["Total_price"].sum().reset_index()
spending_cost

In [ ]:
df["InvoiceDate"].unique()

In [ ]:
reference_date=df["InvoiceDate"].max()+pd.Timedelta(days=1)

In [ ]:
customer_last_purchase=df.groupby("CustomerID")["InvoiceDate"].max().reset_index()
customer_last_purchase

In [ ]:
# How recent did customers bought their purchase
customer_last_purchase["spending_date"]= (reference_date - customer_last_purchase["InvoiceDate"]).dt.days
customer_last_purchase["spending_date"]

In [ ]:
customer_last_purchase[["CustomerID","spending_date"]]

In [ ]:
# Merging Datasets
Customer_data_join=frequent_purchase.merge(spending_cost,on="CustomerID").merge(customer_last_purchase,on="CustomerID")
Customer_data_join

In [ ]:
Customer_data_join.describe()

In [ ]:
df=df[df["Total_price"]>0]
df

In [ ]:
spending_cost=df.groupby("CustomerID")["Total_price"].sum().reset_index()
spending_cost

In [ ]:
spending_cost.rename(columns={"Total_price":"Total_spend"},inplace=True)

In [ ]:
spending_cost

In [ ]:
upper_limit=spending_cost["Total_spend"].quantile(0.95)
spending_cost["Total_spend"]=spending_cost["Total_spend"].apply(lambda x:min(x,upper_limit))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


In [ ]:
Customer_data_join_scaled=scaler.fit_transform(Customer_data_join[["spending_date","InvoiceNo","Total_price"]])
Customer_data_join_scaled

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import matplotlib.pyplot as plt
inertia=[]
for k in range(1,10):
    kmeans=KMeans(n_clusters=k,random_state=42)
    kmeans.fit(Customer_data_join_scaled)
    inertia.append(kmeans.inertia_)
    
plt.plot(range(1,10),inertia,marker='o')
plt.xlabel("Number of clusters (k)")
plt.ylabel("Inertia")
plt.title("Elbow Method")
plt.show()

In [ ]:
kmeans=KMeans(n_clusters=3,random_state=42)
kmeans.fit(Customer_data_join_scaled)
Customer_data_join["Cluster"]=kmeans.labels_

In [ ]:
Customer_data_join.groupby("Cluster")[["Total_price","spending_date","InvoiceNo"]].mean()

In [ ]:
cluster_names={0:'Regular Spending Customers',1:"Inactive Spending Customers",2:"High Spending Customers"}
Customer_data_join["Group"]=Customer_data_join["Cluster"].map(cluster_names)

In [ ]:
Customer_data_join['Cluster'].value_counts()


In [ ]:
Customer_data_join['Group'].value_counts()


In [ ]:
Customer_data_join["Group"]

In [ ]:

import numpy as np 
counts=Customer_data_join["Group"].value_counts()
x=np.arange(len(counts))*2
new_labels=[label.replace(" ", "\n") for label in counts.index]
plt.figure(figsize=(10,6))
bars=plt.bar(counts.index,counts.values,color=['green','orange','blue'])
plt.title("The Number of Customers per Group")
plt.ylabel("Count")
plt.xticks(rotation=0)
for i,v in enumerate(counts.values):
    plt.text(i,v+50,str(v),ha='center')
plt.show()